# Copy "Smart Crop Images" into Train

Based on the Train Assignment

Adaptation of `2020-03-28-TestResults.ipynb` notebook

In [19]:
import numpy as np
import pandas as pd
import os
import s3fs
import boto3
import json
from datetime import datetime
import shutil

In [9]:
pd.set_option('max_colwidth', -1)

# Read Train Reference

In [5]:
SAGEMAKER_REPO_PATH = r'/home/ec2-user/SageMaker/classify-streetview'
df_val_test = pd.read_csv(os.path.join(SAGEMAKER_REPO_PATH, 'label-crops/validation_test_final.csv'))
df_val_test.head()

,image_name,img_id,ground_truth,is_valid_set
0,8475_135_1.jpg,8475,1_null,True
1,8475_135_2.jpg,8475,2_obstacle,True
2,8475_135_3.jpg,8475,1_null,True
3,8475_135_5.jpg,8475,2_obstacle,True
4,8475_135_6.jpg,8475,2_obstacle,True


In [14]:
df_val_test['split_folder'] = np.where(df_val_test['is_valid_set'], 'valid', 'test')
df_val_test['split_folder'].value_counts()

test     958
valid    942
Name: split_folder, dtype: int64

## Create columns for the existing JPG Names and New Names

In [15]:
# New jpg name
df_val_test['original_image_path'] = SAGEMAKER_REPO_PATH +'/all-original-test/' + df_val_test['image_name']
df_val_test['new_image_path'] = SAGEMAKER_REPO_PATH + '/images/' + df_val_test['split_folder'] + '/' + df_val_test['ground_truth'] + '/'+ df_val_test['image_name']
df_val_test.head()

,image_name,img_id,ground_truth,is_valid_set,original_image_path,new_image_path,split_folder
0,8475_135_1.jpg,8475,1_null,True,/home/ec2-user/SageMaker/classify-streetview/all-original-test/8475_135_1.jpg,/home/ec2-user/SageMaker/classify-streetview/images/valid/1_null/8475_135_1.jpg,valid
1,8475_135_2.jpg,8475,2_obstacle,True,/home/ec2-user/SageMaker/classify-streetview/all-original-test/8475_135_2.jpg,/home/ec2-user/SageMaker/classify-streetview/images/valid/2_obstacle/8475_135_2.jpg,valid
2,8475_135_3.jpg,8475,1_null,True,/home/ec2-user/SageMaker/classify-streetview/all-original-test/8475_135_3.jpg,/home/ec2-user/SageMaker/classify-streetview/images/valid/1_null/8475_135_3.jpg,valid
3,8475_135_5.jpg,8475,2_obstacle,True,/home/ec2-user/SageMaker/classify-streetview/all-original-test/8475_135_5.jpg,/home/ec2-user/SageMaker/classify-streetview/images/valid/2_obstacle/8475_135_5.jpg,valid
4,8475_135_6.jpg,8475,2_obstacle,True,/home/ec2-user/SageMaker/classify-streetview/all-original-test/8475_135_6.jpg,/home/ec2-user/SageMaker/classify-streetview/images/valid/2_obstacle/8475_135_6.jpg,valid


In [16]:
df_val_test.to_csv('valid_test_crop_filename.csv')

In [22]:
# Grab a sample of images 
#df_val_test_sample = df_val_test.iloc[1288:1315]

In [17]:
df_sample = df_val_test[0:5]
df_sample

,image_name,img_id,ground_truth,is_valid_set,original_image_path,new_image_path,split_folder
0,8475_135_1.jpg,8475,1_null,True,/home/ec2-user/SageMaker/classify-streetview/all-original-test/8475_135_1.jpg,/home/ec2-user/SageMaker/classify-streetview/images/valid/1_null/8475_135_1.jpg,valid
1,8475_135_2.jpg,8475,2_obstacle,True,/home/ec2-user/SageMaker/classify-streetview/all-original-test/8475_135_2.jpg,/home/ec2-user/SageMaker/classify-streetview/images/valid/2_obstacle/8475_135_2.jpg,valid
2,8475_135_3.jpg,8475,1_null,True,/home/ec2-user/SageMaker/classify-streetview/all-original-test/8475_135_3.jpg,/home/ec2-user/SageMaker/classify-streetview/images/valid/1_null/8475_135_3.jpg,valid
3,8475_135_5.jpg,8475,2_obstacle,True,/home/ec2-user/SageMaker/classify-streetview/all-original-test/8475_135_5.jpg,/home/ec2-user/SageMaker/classify-streetview/images/valid/2_obstacle/8475_135_5.jpg,valid
4,8475_135_6.jpg,8475,2_obstacle,True,/home/ec2-user/SageMaker/classify-streetview/all-original-test/8475_135_6.jpg,/home/ec2-user/SageMaker/classify-streetview/images/valid/2_obstacle/8475_135_6.jpg,valid


In [22]:
for index, row in df_val_test.iterrows():
    original = row['original_image_path']
    target = row['new_image_path']
    try:
        shutil.copyfile(original, target)
    except:
        print(f"could not copy: {row['image_name']}")